In [ ]:
%addconnect name=lake_user, host=XXX.XXX.XXX.XXX

In [ ]:
%chconnect name=lake_user, host=XXX.XXX.XXX.XXX

In [ ]:
%disconnect lake_user

In [ ]:
%connect lake_user, user=data_engineer

In [ ]:
DROP TABLE json_test_ft;

In [ ]:
CREATE FOREIGN TABLE json_test_ft

USING
       (
        LOCATION  ('/az/tdazdemo.blob.core.windows.net/json/customer/')
       );

In [ ]:
select * FROM json_test_ft;

In [ ]:
SELECT Payload.name AS "name", CAST(Payload..address AS JSON(300)) AS Address, TYPE(Address) AS add_type FROM json_test_ft;

In [ ]:
SELECT Payload.name AS "name", Payload..address AS Address, TYPE(Address) AS add_type FROM json_test_ft;

In [ ]:
DROP TABLE stripe_invoices_test;

In [ ]:
CREATE TABLE stripe_invoices_test AS
(
SELECT 
    CAST(Payload."id" AS VARCHAR(255)) AS id_,
    CAST(Payload."tax" AS VARCHAR(255)) AS tax_,  -- Assuming tax can be represented as a string if not null
    CAST(Payload."paid" AS VARCHAR(5)) AS paid_,
    CAST(Payload."lines"."url" AS VARCHAR(255)) AS lines_url_,
    CAST(Payload."lines"."data"[0]."id" AS VARCHAR(255)) AS line_item_id_,
    CAST(Payload."lines"."data"[0]."plan" AS VARCHAR(255)) AS plan_, 
    CAST(Payload."lines"."data"[0]."type" AS VARCHAR(255)) AS type_,
    CAST(Payload."lines"."data"[0]."price"."id" AS VARCHAR(255)) AS price_id_,
    CAST(Payload."lines"."data"[0]."price"."type" AS VARCHAR(255)) AS price_type_,
    CAST(Payload."lines"."data"[0]."price"."active" AS VARCHAR(5)) AS price_active_,
    CAST(Payload."lines"."data"[0]."price"."object" AS VARCHAR(255)) AS price_object_,
    TO_TIMESTAMP(CAST(Payload."lines"."data"[0]."price"."created" AS BIGINT)) AS price_created_,
    CAST(Payload."lines"."data"[0]."price"."product" AS VARCHAR(255)) AS price_product_,
    CAST(Payload."lines"."data"[0]."price"."currency" AS VARCHAR(20)) AS price_currency_,
    CAST(Payload."lines"."data"[0]."price"."livemode" AS VARCHAR(5)) AS price_livemode_,
    CAST(Payload."lines"."data"[0]."price"."nickname" AS VARCHAR(255)) AS price_nickname_, 
    CAST(Payload."lines"."data"[0]."price"."recurring" AS VARCHAR(255)) AS price_recurring_, 
    CAST(Payload."lines"."data"[0]."price"."tiers_mode" AS VARCHAR(255)) AS price_tiers_mode_,
    CAST(Payload."lines"."data"[0]."price"."unit_amount" AS INTEGER) AS price_unit_amount_,
    CAST(Payload."lines"."data"[0]."price"."tax_behavior" AS VARCHAR(255)) AS price_tax_behavior_,
    CAST(Payload."lines"."data"[0]."price"."billing_scheme" AS VARCHAR(255)) AS price_billing_scheme_,
    CAST(Payload."lines"."data"[0]."price"."custom_unit_amount" AS INTEGER) AS price_custom_unit_amount_, 
    CAST(Payload."lines"."data"[0]."price"."transform_quantity" AS VARCHAR(255)) AS price_transform_quantity_,
    CAST(Payload."lines"."data"[0]."price"."unit_amount_decimal" AS DECIMAL(18,2)) AS price_unit_amount_decimal_,
    CAST(Payload."lines"."data"[0]."amount" AS INTEGER) AS amount_,
    CAST(Payload."lines"."data"[0]."object" AS VARCHAR(255)) AS line_item_object_,
    TO_TIMESTAMP(CAST(Payload."lines"."data"[0]."period"."end" AS BIGINT)) AS data_period_end_,
    TO_TIMESTAMP(CAST(Payload."lines"."data"[0]."period"."start" AS BIGINT)) AS data_period_start_,
    CAST(Payload."lines"."data"[0]."invoice" AS VARCHAR(255)) AS invoice_,
    CAST(Payload."lines"."data"[0]."currency" AS VARCHAR(20)) AS data_currency_,
    CAST(Payload."lines"."data"[0]."livemode" AS VARCHAR(5)) AS data_livemode_,
    CAST(Payload."lines"."data"[0]."metadata".* AS JSON) AS metadata_
FROM (
   LOCATION ='/s3/dev-rel-demos.s3.amazonaws.com/demo_data_airbyte_mock/generated_invoices.json') AS stripe_invoices
) WITH DATA;

In [5]:
DROP TABLE stripe_test_ft;

Success: 21 rows affected

In [6]:
CREATE FOREIGN TABLE stripe_test_ft,
EXTERNAL SECURITY Repositories.PubAuth

USING
       (
        LOCATION  ('/s3/dev-rel-demos.s3.amazonaws.com/demo_data_airbyte_mock/generated_invoices.json')
       );

Success: 0 rows affected

In [ ]:
SELECT TOP 5 * FROM stripe_invoices_test;

In [ ]:
SELECT * FROM TD_JSONSHRED(
    ON (select HASHROW(Payload), Payload FROM json_test_ft)
    USING
    ROWEXPR('')
    COLEXPR('name', 'age','hobbies','address')
    RETURNTYPES('VARCHAR(20)', 'INTEGER', 'VARCHAR(500)', 'VARCHAR(500)')
) t;

In [ ]:
--Customer Dimension
SELECT * FROM TD_JSONSHRED(
    ON (select HASHROW(Payload), Payload FROM json_test_ft)
    USING
    ROWEXPR('')
    COLEXPR('name', 'age','address')
    RETURNTYPES('VARCHAR(20)', 'INTEGER','VARCHAR(500)')
) t;

In [ ]:
--Hobbies Dimension
SELECT Column_1 as "CustomerID", Column_2 as "Hobby" FROM TD_JSONSHRED(
    ON (select HASHROW(Payload), Payload FROM json_test_ft)
    USING
    ROWEXPR('hobbies')
    COLEXPR('')
    RETURNTYPES('VARCHAR(500)')
) t;

In [ ]:
SELECT Column_1 AS "CustomerID", "name", street, city, state, zip FROM TD_JSONSHRED
(
    ON (

        SELECT * FROM TD_JSONSHRED(
            ON (select HASHROW(Payload), Payload FROM json_test_ft)
            USING
            ROWEXPR('')
            COLEXPR('name', 'age','address')
            RETURNTYPES('VARCHAR(20)', 'INTEGER','VARCHAR(500)')
        ) AS d1
    )
    USING
        ROWEXPR('')
        COLEXPR('street', 'city', 'state', 'zip')
        RETURNTYPES('VARCHAR(25)', 'VARCHAR(20)', 'VARCHAR(2)', 'VARCHAR(8)')
) AS d2;